# Baseline model (fully connected)

**Dense Network**
- 2 Hidden Layers

**Compiler**

- loss function: binary crossentropy
- optimizer: stochastic gradient descent

**Fit**

- 50 Epochs

**Metrics**

- Accuracy for train: .99 
- Accuracy for val: .81

**Assessment**
- Our model is overfitting

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                245780    
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 147       
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 40        
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 6         
Total params: 245,973
Trainable params: 245,973
Non-trainable params: 0
_________________________________________________________________


## Set up notebook

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
%matplotlib inline

!{sys.executable} -m pip install opencv-python

import time
import scipy
import numpy as np
from PIL import Image
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

np.random.seed(123)

Using TensorFlow backend.


## Scale and Reshape

In [2]:
test_folder = 'data/test'
train_folder = 'data/train'
val_folder = 'data/val'

In [3]:
# get all the data in the directory split/test (180 images), and reshape them
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(test_folder, 
        target_size=(64, 64), batch_size = 624 ) 

# get all the data in the directory split/validation (200 images), and reshape them
val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(val_folder, 
        target_size=(64, 64), batch_size = 16)

# get all the data in the directory split/train (542 images), and reshape them
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(train_folder, 
        target_size=(64, 64), batch_size = 5216)

Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 5216 images belonging to 2 classes.


In [4]:
train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)
val_images, val_labels = next(val_generator)

In [5]:
print(train_labels)

[[1. 0.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [6]:
# Explore your dataset again
m_train = train_images.shape[0]
num_px = train_images.shape[1]
m_test = test_images.shape[0]
m_val = val_images.shape[0]

print ("Number of training samples: " + str(m_train))
print ("Number of testing samples: " + str(m_test))
print ("Number of validation samples: " + str(m_val))
print ("train_images shape: " + str(train_images.shape))
print ("train_labels shape: " + str(train_labels.shape))
print ("test_images shape: " + str(test_images.shape))
print ("test_labels shape: " + str(test_labels.shape))
print ("val_images shape: " + str(val_images.shape))
print ("val_labels shape: " + str(val_labels.shape))

Number of training samples: 5216
Number of testing samples: 624
Number of validation samples: 16
train_images shape: (5216, 64, 64, 3)
train_labels shape: (5216, 2)
test_images shape: (624, 64, 64, 3)
test_labels shape: (624, 2)
val_images shape: (16, 64, 64, 3)
val_labels shape: (16, 2)


In [7]:
train_img = train_images.reshape(train_images.shape[0], -1)
test_img = test_images.reshape(test_images.shape[0], -1)
val_img = val_images.reshape(val_images.shape[0], -1)

print(train_img.shape)
print(test_img.shape)
print(val_img.shape)

(5216, 12288)
(624, 12288)
(16, 12288)


In [8]:
train_y = np.reshape(train_labels[:,0], (5216,1))
test_y = np.reshape(test_labels[:,0], (624,1))
val_y = np.reshape(val_labels[:,0], (16,1))

## Creating X-y

In [9]:
from keras.preprocessing.image import ImageDataGenerator

In [10]:
# Build a baseline fully connected model
from keras import models
from keras import layers
np.random.seed(123)
model = models.Sequential()
model.add(layers.Dense(20, activation='relu', input_shape=(12288,))) # 2 hidden layers
model.add(layers.Dense(7, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [11]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

histoire = model.fit(train_img,
                    train_y,
                    epochs=50,
                    batch_size=16,
                    validation_data=(val_img, val_y), verbose = 1)

Train on 5216 samples, validate on 16 samples
Epoch 1/50
5216/5216 [==============================] - 1s 128us/step - loss: 0.4246 - accuracy: 0.8085 - val_loss: 0.6164 - val_accuracy: 0.7500
Epoch 2/50
5216/5216 [==============================] - 0s 95us/step - loss: 0.2656 - accuracy: 0.8923 - val_loss: 0.4456 - val_accuracy: 0.8125
Epoch 3/50
5216/5216 [==============================] - 0s 81us/step - loss: 0.2024 - accuracy: 0.9225 - val_loss: 0.5388 - val_accuracy: 0.8125
Epoch 4/50
5216/5216 [==============================] - 0s 83us/step - loss: 0.1831 - accuracy: 0.9262 - val_loss: 0.8253 - val_accuracy: 0.6250
Epoch 5/50
5216/5216 [==============================] - 0s 86us/step - loss: 0.1654 - accuracy: 0.9329 - val_loss: 1.0173 - val_accuracy: 0.6250
Epoch 6/50
5216/5216 [==============================] - 0s 83us/step - loss: 0.1469 - accuracy: 0.9431 - val_loss: 0.3601 - val_accuracy: 0.8750
Epoch 7/50
5216/5216 [==============================] - 0s 87us/step - loss: 0.1365

In [12]:
results_train = model.evaluate(train_img, train_y)

5216/5216 [==============================] - 0s 27us/step


In [13]:
results_train

[0.07644408498747857, 0.970475435256958]

In [14]:
results_val = model.evaluate(val_img, val_y)

16/16 [==============================] - 0s 84us/step


In [15]:
results_val

[0.6636800765991211, 0.75]